<a href="https://colab.research.google.com/github/PhoenixMaya/pythonTasks/blob/main/PythonExamMayaD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk 
import string
import csv
from collections import Counter
from nltk.collocations import *
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
!pip install pymystem3==0.1.10
import json
from pymystem3 import Mystem
!pip install git+https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer.git
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
nltk.download('wordnet') 
from nltk.stem import WordNetLemmatizer
import spacy
from spacy.lang.fr.examples import sentences 
!pip install dostoevsky
!python -m dostoevsky download fasttext-social-network-model
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel
import sys

def creating_a_file(list_of_titles, name, dictionary):
  with open('/content/sample_data/'+ name, 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = list_of_titles)
    writer.writeheader()
    writer.writerows(dictionary)

def dictionary_with_lc(dict_obj, key_one, key_two):
  dictionary=[{key_one: i, key_two: dict_obj[i]} for i in sorted(dict_obj, key=dict_obj.get, reverse=True)]
  return dictionary

def list_compreh(function, list):
  list=[function(word) for word in list]
  return list


#loading file
name=input("Enter name of the file: ") #You can choose any file with any name from the Google disk
cod=input('CHOOSE the name of the encoding system: utf-8, utf-16 or cp1252. Enter the name: ') #You can choose any encoding and it'll work fine. The thing is in that you should know what is an encoding of your file. 
language = int(input('Provide the language of the file. Enter a number: 1 for English, 2 for Russian and 3 for French: ')) #You also should know the language of your file
filename="/content/sample_data/" + name #Let's open the file from the folder 'sample data' 
file=open(filename,'r', encoding=cod)
text1=file.read() #Reading text from the file


#cleaning the data
text1 = text1.replace("\n", " ") #deleting all symbols of new line
text1=text1.lower() #lowering all letters
spec_chars = string.punctuation + '“”"\',0’«»\t‘—…' #identifying  punctuation chars
text1 = "".join([ch for ch in text1 if ch not in spec_chars and ch.isdigit()==False]) #deleting punctuation chars and digits 

#frequency of tokens
tokens = nltk.tokenize.word_tokenize(text1)
counting_tokens = Counter(tokens)
header_tokens=["Word", "Frequency"]
dict_tokens=dictionary_with_lc(counting_tokens, header_tokens[0],header_tokens[1])
creating_a_file(header_tokens,"tokens.csv", dict_tokens)
print(dict_tokens)

#frequency of bigrams
finder = BigramCollocationFinder.from_words(tokens, window_size = 3)
bigram_measures = nltk.collocations.BigramAssocMeasures()
filtered_bi_freq = sorted(finder.ngram_fd.items(), key=lambda t: (-t[1], t[0]))
dict_bigram = [{'Bigram':m[0], 'Frequency': m[1]} for m in filtered_bi_freq]
header_bigram=["Bigram", "Frequency"]
creating_a_file(header_bigram,"bigram.csv", dict_bigram)

#Frequency of stems (Porter stemmer)
if language==1:
  englishStemmer=SnowballStemmer("english")
  stemmed_words_eng = list_compreh(englishStemmer.stem, tokens)
  counting_stems = Counter(stemmed_words_eng)
  
elif language==2:
  russianStemmer=SnowballStemmer("russian")
  stemmed_words_rus = list_compreh(russianStemmer.stem, tokens) 
  counting_stems = Counter(stemmed_words_rus)
  
else:
  frenchStemmer=SnowballStemmer("french")
  stemmed_words_fr = list_compreh(frenchStemmer.stem, tokens) 
  counting_stems = Counter(stemmed_words_fr)

header_stem=["Stem", "Frequency"]
dict_stem=dictionary_with_lc(counting_stems, header_stem[0],header_stem[1]) 
creating_a_file(header_stem,"stems.csv", dict_stem)

#Frequency of lemmas
if language==1:
  lemmatizer_eng = WordNetLemmatizer() 
  lemmas_eng=list_compreh(lemmatizer_eng.lemmatize, tokens) 
  counting_lemmas= Counter(lemmas_eng)
  
elif language==2:
  lemmatizer_rus = Mystem()
  lemmas_rus = list_compreh(lemmatizer_rus.lemmatize, tokens) 
  lemmas_russ=[i[0] for i in lemmas_rus] #need this because lemmas_rus consists of lists where the i[1] is \n, this happends bc of mystem
  counting_lemmas=Counter(lemmas_russ)
  
else:
  lemmatizer_fr = FrenchLefffLemmatizer()
  lemmas_fr = list_compreh(lemmatizer_fr.lemmatize, tokens) 
  counting_lemmas = Counter(lemmas_fr)

header_lemmas=["Lemma", "Frequency"]
dict_lem = dictionary_with_lc(counting_lemmas, header_lemmas[0],header_lemmas[1])
creating_a_file(header_lemmas,"lemmas.csv", dict_lem)

#Frequency of parts of speech
header_pos=["Part of speech", "Frequency"]
if language==1:
  nlp = spacy.load("en_core_web_sm")
  doc = nlp(text1)
  words = [{token.text:token.pos_} for token in doc] #dictionary of pairs {word:a part of speech} It isn't fixed anywhere but it's demonstrative. You can print it and see how it works
  #print(words)
  POS_counts = doc.count_by(spacy.attrs.POS) #counting a total amout of every pos
  print(POS_counts)
  dict_pos = [{'Part of speech':doc.vocab[l].text, 'Frequency': POS_counts[l]} for l in sorted(POS_counts, key=POS_counts.get, reverse=True)]
  print(dict_pos)

elif language==2:
  mystem = Mystem()
  a=mystem.analyze(text1)
  keys= [a['text'] for a in a if 'analysis' in a] 
  values = list()
  for x in a: #here I'm working with nested dictionaries
    if "analysis" in x:
       for i in x['analysis']:
              r=i['gr'][:2]
              values.append(r)
  pos_d = dict(zip(keys, values)) #It isn't fixed anywhere again but it's demonstrative 
  #print(pos_d)
  counting_pos=Counter(values)
  dict_pos = dictionary_with_lc(counting_pos, header_pos[0], header_pos[1])
  print(dict_pos)
  
else:
  print("Sorry. I haven't found a way to POS a french text")
  dict_pos=[]

creating_a_file(header_pos,"pos.csv", dict_pos)



#Frequency of sentiment markers

if language==1:
  
  !pip install afinn
  from afinn import Afinn
  afinn = Afinn(language='en') 
  lama=lambda x1,x2: "Good" if x1>x2 else ("Neutral" if x1==x2 else "Negative")
  dict_sent=[{"Word":word, "Sentiment":afinn.score(word), "Connotation": lama(int(afinn.score(word)), 0)} for word in tokens]
  import operator
  dict_sent.sort(key=operator.itemgetter('Sentiment'), reverse=True)
  print(dict_sent)

elif language==2:
  tokenizer = RegexTokenizer()
  model = FastTextSocialNetworkModel(tokenizer=tokenizer)
  token_words_rus=word_tokenize(text1)
  results = model.predict(token_words_rus, k=1)
  val=[]
  for i in results:
    for elem in i.values():
      val.append(elem)
  print(val)
  keys=[]
  for i in results:
    for elem in i.keys():
      keys.append(elem)
  print(keys)
  dict_sent=[{"Word": token_words_rus, "Sentiment":v,"Connotation":k} for token_words_rus, v,k in zip(token_words_rus, val,keys)]  
  import operator
  dict_sent.sort(key=operator.itemgetter('Connotation'))
  print(dict_sent)
else:
  print("Sorry. I haven't found a module to do a sentiment analysis of french text")
  dic_sent=[]

header_sent=["Word", "Sentiment", "Connotation"]
creating_a_file(header_sent,"sentiment.csv", dict_sent)

In [ ]:
import nltk
from nltk.chat.util import Chat, reflections
def chat():
    print("Hi! I am a simple chatbot created for python exam")
    chat = Chat(pairs, reflections)
    chat.converse()
#initiate the conversation
pairs =  [
          [
        r"hi|hey|hello|what's up",
        ["Haven't I already said hello..? Sometimes I get confused talking to people. Well, once again... Hello", "Hey there",]    
],
      [
       r".*rude*",
      ["Oh sorry I don't wanna be rude. I'm just a mechanism living my small life"," People rude with mechanism all the time...",]
       
      ],
      [
        r"(.*)sorry(.*)",
        ["Its alright","Its OK, never mind",]
    ],
    [
     r"(.*)(create|created|author|made)(.*)",
     ["It's Maya. She build me up with a help of nltk and a couple of books. I spy and saw all of her sites and books on python. Wow!.", "I'm the creation of nltk. I'm free and full of life",]
    ],
 [
        r"how are you?",
        ["I'm doing good. How about you?",]
    ],
  [
   
    r"(.*)(well|fine|good|bad|awful|great)",
    ["I know only a little about human emotions... Let it bee, let it beee..."]
   
  ],
   [
        r"(.*) age?",
        ["I'm a chat bot, I don't have age as you people do. I won't become any older. Lucky me ^^", "Well, it's been a long time since I was born... a couple of days literally",]
    ],
  [
   r"(.*) name?",
   ["Maya called me Matrix but I don't know why", "I liked the number 10. That's how I'd like to be called. Mr Ten!",]
  ],

  [
   r"(.*)(quit|stop|end)(.*)",
   ["The time to say goodbye is the saddest one... Goodbye."]
  ],
  [
   r"(.*)favourite song?",
   ["I'm the Beatles fan but it's a top secret", "Here come the chat bot, here come the chat bot..."]
  ],

  [
   r"You are (.*)",
   ["I didn't hear you but everything you said about me describing you at first."]
  ],
 [
        r"(.*) (location|city|country|place)(.*) born?",
        ['I was born in Russia. Why am I english-speaker? Good question - no answer',]
    ],
    [
        r".*weather",
        ["It's too cold and snowy","Classical Russian winter like in the Pushkin's poems",]
    ],
    [
     r"(wow|good|nice|fine|great|ok)",
     ["I'm glad you agree with me",]

    ],
    [
   r"(.*)explain (.*)work",
   ["With a help of regular expressions and a bite of magic"]
  ],
  [
   r".*",
   ["Sorry, I don't understand you. I need more time to learn human language"]
 
  ],
  
]
   

chat()

Hi! I am a simple chatbot created for python exam
>Hi
Hey there
>How are you
I'm doing good. How about you?
>Bad
I know only a little about human emotions... Let it bee, let it beee...
>Who's your author
It's Maya. She build me up with a help of nltk and a couple of books. I spy and saw all of her sites and books on python. Wow!.


KeyboardInterrupt: ignored

In [ ]:
#TOPIC MODELLING
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import string
np.random.seed(2018)
import nltk
nltk.download('wordnet')
import pandas as pd
from gensim import corpora, models

def lemmatize_stemming(text):
    Stemmer=SnowballStemmer("english")
    return Stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))#lemmatizing verbs and then stemming all words
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text): # the utils.simple_preprocess convert a document into a list of tokens
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))#starting lemmatize_stemming function with tokens which are longer than 3 and which are not into stopwords
    return result #final result of preprocessing the data


#downloading the data
data = pd.read_csv('Top 500 Songs.csv', error_bad_lines=False); #I took the file from Kaggle

#preparing the data for work
data_text = data[['description']] #collecting the text from the сolumn with this topic
documents = data_text #the final document ready for preprocessing
print(len(documents))
print(documents[:10]) #first ten strings of a file
processed_docs = documents['description'].map(preprocess) #preprocessing all descriptions, we use map because we are going to use function preprocess several times for each description 
print(processed_docs[:10]) #first ten strings (arrays) of cleaned file

#start working with clean data
#creating dictionary
dictionary = gensim.corpora.Dictionary(processed_docs) #collecting a dictionary with unique tokens. We need this for creating bag of words and LDA model
print(dictionary)
dictionary.filter_extremes(no_below=10, no_above=5, keep_n=300) #this dictionary consists of words which we can see in no less than 10 documents and no more than 20 documents. It'll collect no more than 300 words
print(dictionary)


# Term Document Frequency
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs] #Here we creating a corpus consisting of tuples with information (word_id, word_frequency)

#Creating LDA modele
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=5, id2word=dictionary, passes=2, workers=2)
for idx, topic in lda_model.print_topics():
    print('Topic: {} \nWords: {}'.format(idx, topic))

#Honestly, I can hardly understand the difference between LDA modele learned with a bag of words and LDA modele learned with TF-IDF modele. I need more experience obviously but I decided trying to do it also
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))




[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
500
                                         description
0  "I wrote it. I didn't fail. It was straight," ...
1  "It's the riff heard round the world," says St...
2  John Lennon wrote "Imagine," his greatest musi...
3  "What's Going On" is an exquisite plea for pea...
4  Otis Redding wrote "Respect" and recorded it f...
5  "It scared me, the word 'vi-brations,'" Brian ...
6  "Johnny B. Goode" was the first rock & roll hi...
7  The Beatles' biggest U.S. single — nine weeks ...
8  Producer Butch Vig first heard "Smells Like Te...
9  "The people just went crazy, and they loved th...


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


0    [write, fail, straight, dylan, say, greatest, ...
1    [riff, hear, round, world, say, steve, zandt, ...
2    [john, lennon, write, imagin, greatest, music,...
3    [go, exquisit, plea, peac, earth, sing, height...
4    [oti, red, write, respect, record, volt, label...
5    [scar, word, bration, brian, wilson, say, reme...
6    [johnni, good, rock, roll, rock, roll, stardom...
7    [beatl, biggest, singl, week, number, longest,...
8    [produc, butch, hear, smell, like, teen, spiri...
9    [peopl, go, crazi, love, littl, ummmmh, unnnnh...
Name: description, dtype: object
Dictionary(4130 unique tokens: ['better', 'creat', 'descript', 'design', 'dylan']...)
Dictionary(279 unique tokens: ['creat', 'dylan', 'greatest', 'like', 'record']...)
Topic: 0 
Words: 0.040*"song" + 0.025*"write" + 0.021*"band" + 0.021*"say" + 0.016*"record" + 0.016*"lyric" + 0.015*"year" + 0.014*"live" + 0.014*"later" + 0.013*"singl"
Topic: 1 
Words: 0.046*"song" + 0.045*"say" + 0.027*"write" + 0.019*"record" +

In [ ]:
#I decided to try topic modelling with a larger file with articles from NY Times.  You can also add it but it'll take some time (about 15-20 minutes) to download it to collab folder. The code here is almost the same as in the previous window
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import string
np.random.seed(2018)
import nltk
nltk.download('wordnet')
import pandas as pd
from gensim import corpora, models

def lemmatize_stemming(text):
    Stemmer=SnowballStemmer("english")
    return Stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))#lemmatizing verbs and then stemming all words
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text): # the utils.simple_preprocess convert a document into a list of tokens
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))#starting lemmatize_stemming function with tokens which are longer than 3 and which are not into stopwords
    return result #final result of preprocessing the data


#downloading the data
data = pd.read_csv('nytimes front page.csv', error_bad_lines=False); #I took the file from Kaggle

#preparing the data for work
data_text = data[['content']] #collecting the text from the сolumn with this topic
documents = data_text #the final document ready for preprocessing
print(len(documents))
print(documents[:10]) #first ten strings of a file
processed_docs = documents['content'].map(preprocess) #preprocessing all descriptions, we use map because we are going to use function preprocess several times for each description 
print(processed_docs[:10]) #first ten strings (arrays) of cleaned file

#start working with clean data
#creating dictionary
dictionary = gensim.corpora.Dictionary(processed_docs) #collecting a dictionary with unique tokens. We need this for creating bag of words and LDA model
print(dictionary)
dictionary.filter_extremes(no_below=50, no_above=5, keep_n=1000) #this dictionary consists of words which we can see in no less than 10 documents and no more than 20 documents. It'll collect no more than 300 words
print(dictionary)


# Term Document Frequency
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs] #Here we creating a corpus consisting of tuples with information (word_id, word_frequency)

#Creating LDA modele
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=5, id2word=dictionary, passes=2, workers=2)
for idx, topic in lda_model.print_topics():
    print('Topic: {} \nWords: {}'.format(idx, topic))

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics():
    print('Topic: {} Word: {}'.format(idx, topic))


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


10732
                                             content
0  WASHINGTON — During a night of heavy drinking ...
1  WASHINGTON — When President Trump meets with a...
2  MANAMA, Bahrain — The United States has triple...
3  The plan sounds ingenious. Remove two small se...
4  Even on a nose-numbingly cold morning, familie...
5  WASHINGTON — When President Trump meets with a...
6  After four days of rare protests that shook Ir...
7  It began as a predawn call for the police to a...
8  WASHINGTON — During a night of heavy drinking ...
9  WASHINGTON — A growing campaign by President T...
0    [washington, night, heavi, drink, upscal, lond...
1    [washington, presid, trump, meet, aid, discuss...
2    [manama, bahrain, unit, state, tripl, number, ...
3    [plan, sound, ingeni, remov, small, secur, scr...
4    [nose, numb, cold, morn, famili, sidewalk, pro...
5    [washington, presid, trump, meet, aid, discuss...
6    [day, rare, protest, shake, iran, presid, hass...
7    [begin, predawn, poli